In [1]:
setwd("/import/home/share/zw/HSS")
getwd()

[1] "/import/home/share/zw/HSS"

In [2]:
# install.packages("devtools")
# devtools::install_github("statwangz/HSS")
library(HSS)

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: genio



In [3]:
trait_info

Trait,PMID,Description
<chr>,<dbl>,<chr>
Angina,31427789,Vascular/heart problems diagnosed by doctor: Angina
BMI,31427789,Body Mass Index
Depression,29942085,Depressive symptoms
Hair,31427789,"Hair colour (natural, before greying): Black"
HBP,31427789,Vascular/heart problems diagnosed by doctor: High blood pressure
Height,31427789,Standing height
Income,31427789,Average total household income before tax
Intelligence,31427789,Fluid intelligence test - Fluid intelligence score
MDD,31427789,"Diagnoses - secondary ICD10: F32 Major depressive disorder, single episode"


In [4]:
# LD score regression
load("out/ldsc.rda")
ldsc %>%
  select(-Trait) %>%
  round(digits = 3) %>%
  cbind(Trait = ldsc$Trait, .)

Trait,h2,se
<chr>,<dbl>,<dbl>
Angina,0.022,0.002
BMI,0.238,0.008
Depression,0.051,0.003
Hair,0.064,0.009
HBP,0.125,0.005
Height,0.457,0.019
Income,0.069,0.004
Intelligence,0.195,0.010
MDD,0.019,0.002


In [5]:
# XPASS
load("out/xpass.rda")
xpass %>%
  select(-Trait) %>%
  round(digits = 3) %>%
  cbind(Trait = xpass$Trait, .)

Trait,h2,se,h2_cov,se_cov,h2_cov_5pc,se_cov_5pc,h2_cov_10pc,se_cov_10pc
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Angina,0.015,0.001,0.023,0.001,0.024,0.001,0.025,0.001
BMI,0.148,0.004,0.237,0.007,0.245,0.007,0.252,0.007
Depression,0.030,0.001,0.047,0.002,0.049,0.002,0.050,0.002
Hair,0.083,0.006,0.133,0.010,0.138,0.010,0.142,0.011
HBP,0.080,0.003,0.128,0.004,0.132,0.004,0.136,0.004
Height,0.341,0.013,0.544,0.020,0.563,0.021,0.578,0.021
Income,0.048,0.002,0.077,0.003,0.080,0.003,0.082,0.003
Intelligence,0.118,0.005,0.188,0.008,0.195,0.008,0.200,0.009
MDD,0.008,0.001,0.013,0.001,0.013,0.001,0.014,0.001
